In [9]:
!pip3 install pyspark



Defaulting to user installation because normal site-packages is not writeable


In [1]:
from pyspark.context import SparkContext

In [2]:
sc = SparkContext('local', 'test')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/17 06:05:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc

<SparkContext master=local appName=test>

In [4]:
from operator import add

In [6]:
# Reading file into RDD
d1 = sc.textFile("./d1.txt")

In [7]:
d1_freq = d1.map(lambda x: x.lower()).flatMap(lambda x: x.split(' ')).map(lambda x:(x,1)) \
             .reduceByKey(lambda x, y: x + y).map(lambda x:(x[0],'d1',x[1])) 

In [8]:
d1_freq.take(3)

[('china', 'd1', 5), ('officially', 'd1', 1), ('the', 'd1', 7)]

In [10]:
# Reading file into RDD
d2 = sc.textFile("./d2.txt")

In [11]:
d2_freq = d2.map(lambda x: x.lower()).flatMap(lambda x: x.split(' ')).map(lambda x:(x,1)) \
             .reduceByKey(lambda x, y: x + y).map(lambda x:(x[0],'d2',x[1])) 

In [12]:
d2_freq.take(5)

[('india', 'd2', 4),
 ('officially', 'd2', 1),
 ('the', 'd2', 17),
 ('republic', 'd2', 1),
 ('of', 'd2', 3)]

In [13]:
# Reading file into RDD
d3 = sc.textFile("./d3.txt")

In [14]:
d3_freq = d3.map(lambda x: x.lower()).flatMap(lambda x: x.split(' ')).map(lambda x:(x,1)) \
             .reduceByKey(lambda x, y: x + y).map(lambda x:(x[0],'d3',x[1])) 

In [15]:
d3_freq.take(3)

[('relations', 'd3', 2), ('between', 'd3', 2), ('india', 'd3', 7)]

In [16]:
# Reading file into RDD
d4 = sc.textFile("./d4.txt")

In [17]:
d4_freq = d4.map(lambda x: x.lower()).flatMap(lambda x: x.split(' ')).map(lambda x:(x,1)) \
             .reduceByKey(lambda x, y: x + y).map(lambda x:(x[0],'d4',x[1]))

In [18]:
d4_freq.take(3)

[('the', 'd4', 26), ('india', 'd4', 11), ('us', 'd4', 9)]

In [19]:
# Reading file into RDD
d5 = sc.textFile("./d5.txt")

In [20]:
d5_freq = d5.map(lambda x: x.lower()).flatMap(lambda x: x.split(' ')).map(lambda x:(x,1)) \
             .reduceByKey(lambda x, y: x + y).map(lambda x:(x[0],'d5',x[1]))

In [21]:
d5_freq.take(3)

[('russia', 'd5', 5), ('also', 'd5', 2), ('officially', 'd5', 1)]

In [22]:
# Reading file into RDD
d6 = sc.textFile("./d6.txt")

In [23]:
d6_freq = d6.map(lambda x: x.lower()).flatMap(lambda x: x.split(' ')).map(lambda x:(x,1)) \
             .reduceByKey(lambda x, y: x + y).map(lambda x:(x[0],'d6',x[1]))

In [24]:
d6_freq.take(3)

[('the', 'd6', 16), ('united', 'd6', 4), ('states', 'd6', 7)]

In [25]:
# Reading file into RDD
d7 = sc.textFile("./d7.txt")

In [26]:
d7_freq = d7.map(lambda x: x.lower()).flatMap(lambda x: x.split(' ')).map(lambda x:(x,1)) \
             .reduceByKey(lambda x, y: x + y).map(lambda x:(x[0],'d7',x[1]))

In [27]:
d7_freq.take(3)

[('russia', 'd7', 6), ('united', 'd7', 4), ('states', 'd7', 5)]

In [28]:
combined_freq = d1_freq.union(d2_freq).union(d3_freq).union(d4_freq).union(d5_freq).union(d6_freq).union(d7_freq)

In [29]:
combined_freq.take(5)

[('china', 'd1', 5),
 ('officially', 'd1', 1),
 ('the', 'd1', 7),
 ('people', 'd1', 2),
 ('republic', 'd1', 3)]

In [30]:
inverted_index = combined_freq.map(lambda x: (x[0],[(x[1],x[2])]))
inverted_index.take(3)

[('china', [('d1', 5)]), ('officially', [('d1', 1)]), ('the', [('d1', 7)])]

In [31]:
inverted_index_red_sorted = inverted_index.sortBy(lambda x:(x[1])).reduceByKey(add).sortByKey(ascending=True)
inverted_index_red_sorted.take(3)

[('', [('d3', 1)]),
 ('a',
  [('d1', 4),
   ('d2', 2),
   ('d3', 5),
   ('d4', 5),
   ('d5', 3),
   ('d6', 3),
   ('d7', 6)]),
 ('about', [('d4', 1)])]

In [32]:
inverted_index_final =inverted_index_red_sorted.map(lambda x: x[0] + '#' + ';'.join(['%s:%d' % (i[0], i[1]) for i in x[1]]))

In [33]:
inverted_index_final.collect()

['#d3:1',
 'a#d1:4;d2:2;d3:5;d4:5;d5:3;d6:3;d7:6',
 'about#d4:1',
 'across#d5:2',
 'actors#d4:1',
 'addition#d2:1',
 'administrative#d1:1',
 'affect#d4:1',
 'affects#d4:1',
 'afghanistan#d4:1',
 'after#d3:2;d7:1',
 'agenda#d7:1',
 'agree#d7:1',
 'air#d7:1',
 'aircraft#d7:1',
 'airlines#d7:1',
 'alaska#d5:1;d6:1',
 'all#d3:1',
 'also#d1:1;d4:1;d5:2;d6:2',
 'although#d4:2',
 'america#d4:1;d6:4;d7:1',
 'an#d3:2;d4:1;d6:1;d7:1',
 'and#d1:3;d2:7;d3:13;d4:12;d5:6;d6:11;d7:8',
 'andaman#d2:1',
 'announced#d7:1',
 'anti#d7:1',
 'any#d4:1',
 'arabian#d2:1',
 'archipelago#d6:1',
 'arctic#d5:1',
 'are#d3:1;d4:3;d6:1',
 'area#d2:1;d5:1;d6:1;d7:1',
 'armed#d3:1',
 'arms#d7:1',
 'as#d1:1;d3:2;d4:3;d5:1;d6:1;d7:3',
 'asia#d1:1;d2:1;d4:1;d5:1',
 'asian#d3:1;d4:2',
 'at#d4:2',
 'attempt#d4:2',
 'attempted#d7:1',
 'attracted#d7:1',
 'autonomous#d1:1',
 'azerbaijan#d5:1',
 'back#d7:1',
 'backyard#d4:1',
 'bangladesh#d2:1;d3:2',
 'bay#d2:1',
 'be#d4:1',
 'became#d7:2',
 'because#d4:1',
 'been#d3:3;d7:1',
